In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

In [3]:
spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/12/23 21:25:44 WARN Utils: Your hostname, DESKTOP-H50ARNK resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/23 21:25:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 21:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv("dataset.csv",header=True,inferSchema=True)

In [4]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [5]:
df.show(2)

+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|   type| amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1|PAYMENT|9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1|PAYMENT|1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
only showing top 2 rows



In [5]:
df = df.drop('isFraud','isFlaggedFraud')

In [7]:
df.show(2)

+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|   type| amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
|   1|PAYMENT|9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|
|   1|PAYMENT|1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
only showing top 2 rows



In [8]:
df.groupBy('step').count().show(3)

+----+-----+
|step|count|
+----+-----+
|  12|36153|
|   1| 2708|
|  13|37515|
+----+-----+
only showing top 3 rows



In [ ]:
%%time
steps = df.select('step').distinct().collect()
for step in steps[:]:
    _df = df.where(f"step = {step[0]}")
    _df.coalesce(1).write.mode('append').option('header','true').csv('dataset')

Somethings killed this progress in my computer. And there was many warnings. Afterward, I tried to optimize this code block without many methods, with same features.

In [23]:
steps = df.select('step').distinct().collect()

for step in steps[:]:
    result = df.where(f"step = {step[0]}")
    result.coalesce(1).write.csv('dataset', header=True, mode="append")
    


In [7]:
!cd dataset && ls

_SUCCESS
part-00000-00d5bc0e-f098-4d5c-82ed-85e8a8002daa-c000.csv
part-00000-014f8b1c-88f1-4bba-a41c-2264d85caa10-c000.csv
part-00000-0175ba35-29e6-4ee2-a92d-424e0c96b485-c000.csv
part-00000-01d3c2d5-cccd-4b4a-acbe-658e8df54bb8-c000.csv
part-00000-01e4abd1-7c69-491f-a75d-58e07a10376a-c000.csv
part-00000-020c893b-2958-46eb-ab48-0ccdb586b322-c000.csv
part-00000-02201920-ca44-44f4-9824-7355209ead55-c000.csv
part-00000-02214e7c-9ff0-41cf-aca8-1e5725c9b6ad-c000.csv
part-00000-023a33b5-ecdf-4a5d-866d-d4d29614b113-c000.csv
part-00000-0250995a-d86d-4925-8b73-84d787d24518-c000.csv
part-00000-029b8e2b-87da-4d3e-99c5-b3a82a910e47-c000.csv
part-00000-0324d9ec-08bc-49a9-a92f-b14f0209b4ae-c000.csv
part-00000-03a8240f-a329-4c1c-aba9-1a3fa5d71524-c000.csv
part-00000-0409f890-ad9b-4bdf-b7e0-bbfa6f654fa0-c000.csv
part-00000-04a8b864-abb2-41de-abde-a8bda0bfa09c-c000.csv
part-00000-04d1920b-5af5-4d23-af5c-fa220a859784-c000.csv
part-00000-04f2cb8c-2477-4e86-bc53-041f77a60789-c000.csv
part-00000-04f4c30f-72

In [11]:
part = spark.read.csv("dataset/part-00000-00d5bc0e-f098-4d5c-82ed-85e8a8002daa-c000.csv", header=True, inferSchema=True)

In [12]:
part.groupBy("step").count().show()

+----+-----+
|step|count|
+----+-----+
|  48| 5693|
+----+-----+



In [13]:
dataschema = part.schema

In [14]:
dataschema

StructType([StructField('step', IntegerType(), True), StructField('type', StringType(), True), StructField('amount', DoubleType(), True), StructField('nameOrig', StringType(), True), StructField('oldbalanceOrg', DoubleType(), True), StructField('newbalanceOrig', DoubleType(), True), StructField('nameDest', StringType(), True), StructField('oldbalanceDest', DoubleType(), True), StructField('newbalanceDest', DoubleType(), True), StructField('isFraud', IntegerType(), True), StructField('isFlaggedFraud', IntegerType(), True)])

In [19]:
streaming = (
    spark.readStream.schema(dataschema)
    .option("maxFilesPerTrigger", 1)
    .csv("dataset")
)

In [20]:
dest_count = streaming.groupBy("nameDest").count().orderBy(desc("count"))

In [26]:
activityQuery = (
    dest_count.writeStream.queryName("dest_counts")
    .format("memory")
    .outputMode("complete")
    .start()
)

# include this in production:
activityQuery.awaitTermination()

import time

for x in range(50):
    _df = spark.sql(
        "SELECT * FROM dest_counts WHERE nameDest != 'nameDest' AND count >= 2;"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)


24/12/23 21:55:29 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-28476d20-c19b-45c8-aa75-ef938c0a2f53. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/23 21:55:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


IllegalArgumentException: Cannot start query with name dest_counts as a query with that name is already active in this SparkSession

In [22]:
spark.streams.active[0].isActive

True

In [23]:
activityQuery.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [27]:
activityQuery.stop()

24/12/23 21:57:40 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 92, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@16dd1751] is aborting.
24/12/23 21:57:40 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 92, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@16dd1751] aborted.
